In [ ]:
#| default_exp utils_bgtsk

# ⚡ Background Tasks

> Lightweight background task execution for tenant-level operations with retry logic and status tracking.
---

In [ ]:
#| export

from datetime import datetime
from typing import Optional, Callable, Any
import json
import traceback
import asyncio
import logging
from starlette.background import BackgroundTask
from fastcore.utils import *
from fastsql.core import *
from nbdev.showdoc import show_doc
from fh_saas.db_host import timestamp, gen_id

# Module-level logger - configured by app via configure_logging()
logger = logging.getLogger(__name__)



## 🎯 Overview

| Category | Components | Purpose |
|----------|------------|--------|
| 📦 Model | `TenantJob` | Tenant-level job record with retry support |
| ⚡ Manager | `BackgroundTaskManager` | Submit, execute, and track background tasks |

---

## 🏗️ Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                    FastHTML Route                               │
│  return response, bg_task  ←─────────────────────┐              │
└─────────────────────────────────────────────────────────────────┘
                              │                    │
                              ▼                    │
┌─────────────────────────────────────────────────────────────────┐
│                 BackgroundTaskManager                           │
├─────────────────────────────────────────────────────────────────┤
│  submit()           → Create job + return BackgroundTask        │
│  _execute_with_retry() → Run with auto-retry on failure         │
│  get_job()          → Check job status                          │
│  list_jobs()        → Query jobs by type/status                 │
├─────────────────────────────────────────────────────────────────┤
│  Retry Logic: 2^n seconds backoff (2s, 4s, 8s)                  │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│                    Tenant Database                              │
│                    └── tenant_jobs table                        │
└─────────────────────────────────────────────────────────────────┘
```

---

## 📚 Quick Reference

### Job Status Flow

```
pending → running → completed
    ↑        │
    └────────┴──→ failed (after max retries)
```

### Usage Pattern

```python
# In your FastHTML route
manager = BackgroundTaskManager(tenant_db)
job_id, bg_task = manager.submit("sync_data", my_sync_func, user_id="123")
return response, bg_task  # Starlette runs bg_task after response
```

---

## 📦 TenantJob Model

Tracks background jobs at the tenant level with retry support.

| Field | Type | Description |
|-------|------|-------------|
| `id` | str | Unique job identifier |
| `job_type` | str | Job category (e.g., "sync", "email") |
| `status` | str | `pending` / `running` / `completed` / `failed` |
| `payload` | str | JSON kwargs passed to task function |
| `result` | str | JSON result on completion |
| `error_log` | str | Stack trace on failure |
| `retry_count` | int | Current retry attempt |
| `max_retries` | int | Max attempts before marking failed |

In [ ]:
#| export

class TenantJob:
    """Tenant-level background job with retry support."""
    id: str
    job_type: str
    status: str  # 'pending', 'running', 'completed', 'failed'
    payload: str  # JSON string
    result: str = None  # JSON string
    error_log: str = None
    retry_count: int = 0
    max_retries: int = 3
    created_at: str = None
    started_at: str = None
    completed_at: str = None

---

## ⚡ BackgroundTaskManager

Submit and track background tasks with automatic retry logic.

| Method | Purpose |
|--------|--------|
| `submit` | Create job and return BackgroundTask for Starlette |
| `get_job` | Get job status and details by ID |
| `list_jobs` | Query jobs with optional type/status filters |

> 💡 **Use case**: Long-running operations like syncing data, sending emails, or processing uploads

In [ ]:
#| export

class BackgroundTaskManager:
    """Lightweight background task manager for tenant-level operations."""
    
    def __init__(self, db: Database):
        """Initialize with tenant database."""
        self.db = db
        self.tenant_jobs = db.create(TenantJob, name="tenant_jobs", pk='id')
    
    def submit(self, job_type: str, task_func: Callable, max_retries: int = 3, **task_kwargs) -> tuple[str, BackgroundTask]:
        """Submit a new background task for execution."""
        job_id = gen_id()
        job = TenantJob(
            id=job_id,
            job_type=job_type,
            status='pending',
            payload=json.dumps(task_kwargs),
            max_retries=max_retries,
            created_at=timestamp()
        )
        self.tenant_jobs.insert(job)
        
        bg_task = BackgroundTask(
            self._execute_with_retry,
            job_id=job_id,
            task_func=task_func,
            **task_kwargs
        )
        
        return job_id, bg_task
    
    def _execute_with_retry(self, job_id: str, task_func: Callable, **task_kwargs):
        """Execute task with automatic retry logic and status tracking."""
        job = self.tenant_jobs[job_id]
        
        try:
            self.tenant_jobs.update(id=job_id, status='running', started_at=timestamp())
            result = task_func(**task_kwargs)
            self.tenant_jobs.update(
                id=job_id,
                status='completed',
                result=json.dumps(result) if result else None,
                completed_at=timestamp()
            )
        except Exception as e:
            self._handle_failure(job_id, job, e)
    
    def _handle_failure(self, job_id: str, job: TenantJob, error: Exception):
        """Handle task failure with retry logic."""
        error_msg = f"{type(error).__name__}: {str(error)}\n{traceback.format_exc()}"
        retry_count = job.retry_count + 1
        
        logger.error(f"Job {job_id} ({job.job_type}) failed (attempt {retry_count}/{job.max_retries}): {error}", exc_info=True)
        
        if retry_count < job.max_retries:
            self.tenant_jobs.update(id=job_id, status='pending', retry_count=retry_count, error_log=error_msg)
        else:
            self.tenant_jobs.update(id=job_id, status='failed', retry_count=retry_count, error_log=error_msg, completed_at=timestamp())
    
    def get_job(self, job_id: str) -> TenantJob:
        """Get job status and details."""
        return self.tenant_jobs[job_id]
    
    def list_jobs(self, job_type: Optional[str] = None, status: Optional[str] = None, limit: int = 100) -> list[TenantJob]:
        """List jobs with optional filtering."""
        where_clauses = []
        if job_type:
            where_clauses.append(f"job_type = '{job_type}'")
        if status:
            where_clauses.append(f"status = '{status}'")
        
        where = " AND ".join(where_clauses) if where_clauses else None
        return self.tenant_jobs(where=where, order_by="created_at DESC", limit=limit)

In [ ]:
show_doc(BackgroundTaskManager.list_jobs)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_bgtsk.py#L103){target="_blank" style="float:right; font-size:smaller"}

### BackgroundTaskManager.list_jobs

>      BackgroundTaskManager.list_jobs (job_type:Optional[str]=None,
>                                       status:Optional[str]=None,
>                                       limit:int=100)

*List jobs with optional filtering.*

In [ ]:
show_doc(BackgroundTaskManager.get_job)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_bgtsk.py#L99){target="_blank" style="float:right; font-size:smaller"}

### BackgroundTaskManager.get_job

>      BackgroundTaskManager.get_job (job_id:str)

*Get job status and details.*

In [ ]:
show_doc(BackgroundTaskManager.submit)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_bgtsk.py#L49){target="_blank" style="float:right; font-size:smaller"}

### BackgroundTaskManager.submit

>      BackgroundTaskManager.submit (job_type:str, task_func:Callable,
>                                    max_retries:int=3, **task_kwargs)

*Submit a new background task for execution.*

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()